In [2]:
"""
Code to download ERA5 reanalysis data at 6hr intervals as netcdfs using the cds api.

Select variables then provide inputs before running.
Make sure target directory exists before downloading.

***  Make sure username is changed otherwise you'll overwrite my files I've downloaded :) ***

For more info see:
https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview
https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels?tab=overview

Look here for active requests:
https://cds.climate.copernicus.eu/cdsapp#!/yourrequests?tab=form
"""

import cdsapi
import os

In [11]:
# SELECT VARIABLES (Only select ones on same model level together)

# SINGLE LEVEL
variable_names = []
#variable_names.append('mean_sea_level_pressure')
# variable_names.append('10m_u_component_of_wind')
# variable_names.append('10m_v_component_of_wind')
# variable_names.append('sea_ice_cover')
# variable_names.append('total_cloud_cover')
#variable_names.append('surface_latent_heat_flux')
#variable_names.append('surface_net_solar_radiation')
#variable_names.append('surface_net_thermal_radiation')
#variable_names.append('surface_sensible_heat_flux')
#variable_names.append('surface_solar_radiation_downwards')
#variable_names.append('surface_thermal_radiation_downwards')
#variable_names.append('low_cloud_cover')
#variable_names.append('high_cloud_cover')
#variable_names.append('medium_cloud_cover')

# variable_names.append('2m_temperature')
# variable_names.append('significant_height_of_combined_wind_waves_and_swell')
# variable_names.append('significant_height_of_wind_waves')
# variable_names.append('significant_height_of_total_swell')
# variable_names.append('maximum_individual_wave_height')
# variable_names.append('peak_wave_period')
# variable_names.append('mean_period_of_wind_waves')
# variable_names.append('mean_wave_direction')
# variable_names.append('ocean_surface_stress_equivalent_10m_neutral_wind_speed')
# variable_names.append('ocean_surface_stress_equivalent_10m_neutral_wind_direction')
product_source = 'reanalysis-era5-single-levels'

# PRESSURE LEVELS (Mostly on different levels so download one variable at a time)
#variable_names = ['geopotential']
#variable_names = ['vertical_velocity']
# variable_names = ['temperature']
# pressure_list = ['500'] #pressures in mb of levels to be downloaded
# product_source = 'reanalysis-era5-pressure-levels'

# POTENTIAL VORTICITY LEVEL
#have to go through different method. Use mars request
#https://confluence.ecmwf.int/display/CKB/How+to+download+ERA5

In [12]:
# PROVIDE INPUTS
product_type = 'reanalysis' #'reanalysis' or 'ensemble_spread'
start_year = 2018
end_year = 2018 # end_year is included in download
# month_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'] # note formate of leading zero for single digit months.
month_list = ['07', '08']
time_list = ['00:00', '06:00', '12:00', '18:00']
# Area: North, West (-ve), South (-ve), East. Default: global
area_list  = [90, -180, 60, 180] 
# Latitude/longitude grid: east-west (longitude) and north-south resolution (latitude).
# Default: 0.25 x 0.25 for reanalysis at least. Might be coarser for ensemble members/spread
grid_list =  [0.25, 0.25]
username = 'vcooper' #change to your own

In [13]:
# Get year list
year_list = []
for i in range(start_year, end_year+1):
    year_list.append(str(i))

In [14]:
# Get day list
day_list = []
for i in range(1, 31+1):
    day_list.append(str(i).zfill(2))

In [15]:
# Put inputs into dictionary
cdict = {
        'product_type' : product_type,
        'year': year_list,
        'day': day_list,
        'area' : area_list,
        'grid' : grid_list,
        'time' : time_list,
        'format' : 'netcdf',
        }

if product_source == 'reanalysis-era5-pressure-levels':
    cdict['pressure_level'] = pressure_list

In [16]:
for variable_name in variable_names:
    print(variable_name)

10m_u_component_of_wind
10m_v_component_of_wind
sea_ice_cover
2m_temperature
significant_height_of_combined_wind_waves_and_swell
significant_height_of_wind_waves
significant_height_of_total_swell
maximum_individual_wave_height
peak_wave_period
mean_period_of_wind_waves
mean_wave_direction
ocean_surface_stress_equivalent_10m_neutral_wind_speed
ocean_surface_stress_equivalent_10m_neutral_wind_direction


In [19]:
# Download data for each variable for each month
for variable_name in variable_names:
    cdict['variable'] = variable_name
    for month in month_list:
        cdict['month'] = month
        
        # Set file name
        mstr = 'months=' + month
        ystr = '_years=' + str(start_year) + 'to' + str(end_year)
        astr = '_minlat=' + str(area_list[2])
        gstr = '_grid=' + str(grid_list[0]) + 'x' + str(grid_list[1])

        if product_source == 'reanalysis-era5-single-levels':
            f_name = mstr  + ystr + astr + gstr + variable_name + '.nc'
        elif product_source == 'reanalysis-era5-pressure-levels':
            if len(pressure_list) == 1:
                pstr = '_plevel=' + pressure_list[0] + 'hPa'
            else:
                pstr = '_plevel=many'
            f_name = mstr  + ystr + astr + pstr + gstr + variable_name + '.nc'

        if product_type == 'reanalysis':
            target = os.path.join('/glade', 'scratch', username, 'ERA5', 'native', f_name)
        elif product_type == 'ensemble_spread':
            target = os.path.join('/home', 'disk', 'sipn', username, 'ERA5', 'native', 'e_spread', f_name)
        print(target)
        c = cdsapi.Client()
        c.retrieve(product_source, cdict, target)

/glade/scratch/vcooper/ERA5/native/months=07_years=2018to2018_minlat=60_grid=0.25x0.2510m_u_component_of_wind.nc


2020-08-31 13:17:04,656 INFO Welcome to the CDS
2020-08-31 13:17:04,657 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2020-08-31 13:17:05,219 INFO Request is completed
2020-08-31 13:17:05,220 INFO Downloading http://136.156.132.198/cache-compute-0003/cache/data4/adaptor.mars.internal-1598901196.280285-17393-11-2bd02a84-9737-4720-8470-441dbe029921.nc to /glade/scratch/vcooper/ERA5/native/months=07_years=2018to2018_minlat=60_grid=0.25x0.2510m_u_component_of_wind.nc (41.2M)
2020-08-31 13:17:10,054 INFO Download rate 8.5M/s   


/glade/scratch/vcooper/ERA5/native/months=08_years=2018to2018_minlat=60_grid=0.25x0.2510m_u_component_of_wind.nc


2020-08-31 13:17:11,491 INFO Welcome to the CDS
2020-08-31 13:17:11,492 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2020-08-31 13:17:12,086 INFO Request is queued
2020-08-31 13:17:17,202 INFO Request is running
2020-08-31 13:17:33,612 INFO Request is completed
2020-08-31 13:17:33,612 INFO Downloading http://136.156.132.210/cache-compute-0005/cache/data0/adaptor.mars.internal-1598901416.166178-28966-28-324d1f04-7ce2-4322-af03-f80d4c6477bd.nc to /glade/scratch/vcooper/ERA5/native/months=08_years=2018to2018_minlat=60_grid=0.25x0.2510m_u_component_of_wind.nc (41.2M)
2020-08-31 13:17:39,386 INFO Download rate 7.1M/s   


/glade/scratch/vcooper/ERA5/native/months=07_years=2018to2018_minlat=60_grid=0.25x0.2510m_v_component_of_wind.nc


...

2020-08-31 13:31:11,929 INFO Welcome to the CDS
2020-08-31 13:31:11,930 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2020-08-31 13:31:12,538 INFO Request is queued
2020-08-31 13:31:13,658 INFO Request is running
2020-08-31 13:31:45,567 INFO Request is completed
2020-08-31 13:31:45,568 INFO Downloading http://136.156.132.198/cache-compute-0003/cache/data9/adaptor.mars.internal-1598902254.495113-24365-9-946827d9-8c43-45c4-9eda-f6bca0739ca2.nc to /glade/scratch/vcooper/ERA5/native/months=08_years=2018to2018_minlat=60_grid=0.25x0.25ocean_surface_stress_equivalent_10m_neutral_wind_speed.nc (41.2M)
2020-08-31 13:31:54,436 INFO Download rate 4.6M/s   


/glade/scratch/vcooper/ERA5/native/months=07_years=2018to2018_minlat=60_grid=0.25x0.25ocean_surface_stress_equivalent_10m_neutral_wind_direction.nc


2020-08-31 13:31:55,364 INFO Welcome to the CDS
2020-08-31 13:31:55,365 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2020-08-31 13:31:56,499 INFO Request is queued
2020-08-31 13:31:57,620 INFO Request is running
2020-08-31 13:32:18,018 INFO Request is completed
2020-08-31 13:32:18,019 INFO Downloading http://136.156.133.25/cache-compute-0008/cache/data7/adaptor.mars.internal-1598902297.9443924-2136-1-9acd33fd-14b9-4468-bffa-c271b189b5ff.nc to /glade/scratch/vcooper/ERA5/native/months=07_years=2018to2018_minlat=60_grid=0.25x0.25ocean_surface_stress_equivalent_10m_neutral_wind_direction.nc (41.2M)
2020-08-31 13:32:24,104 INFO Download rate 6.8M/s   


/glade/scratch/vcooper/ERA5/native/months=08_years=2018to2018_minlat=60_grid=0.25x0.25ocean_surface_stress_equivalent_10m_neutral_wind_direction.nc


2020-08-31 13:32:24,875 INFO Welcome to the CDS
2020-08-31 13:32:24,876 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2020-08-31 13:32:25,755 INFO Request is queued
2020-08-31 13:32:26,875 INFO Request is running
2020-08-31 13:32:47,275 INFO Request is completed
2020-08-31 13:32:47,276 INFO Downloading http://136.156.133.37/cache-compute-0011/cache/data9/adaptor.mars.internal-1598902327.1880836-31697-35-a35463c4-c67e-4e0e-8cc6-8af8a93f9cdc.nc to /glade/scratch/vcooper/ERA5/native/months=08_years=2018to2018_minlat=60_grid=0.25x0.25ocean_surface_stress_equivalent_10m_neutral_wind_direction.nc (41.2M)
2020-08-31 13:32:51,004 INFO Download rate 11.1M/s  


In [20]:
#Print done?
print('Done')

Done
